# 多重继承



# Mixin

我们知道多重继承是危险的,很容易造成继承混乱,如何解决这个问题呢,就是使用mixin.原则上,应该只在使用Mixin组件制作工具时进行多重继承.


mixin是一个行为的集合,是受限制的多重继承.mixin定义的这个行为可以被加到任意class里，然而在一些情况下，使用mix-in的类，可以要求宿主满足一些协议(contract),这个协议可以是属性也可以是方法. 如果有协议要求的话，协议应该是被声明在mixin内的.这样更容易复用.


Mixin是一种非常谨慎的多重继承用法,它的特点是:

+ Mixin 类是单一职责的
+ Mixin 类对宿主类一无所知
+ 不存在超类方法调用（super）以避免引入 MRO 查找顺序问题

## 例:把内存中的python对象转换为字典形式

In [12]:
class ToDictMixin:
    def to_dict(self):
        return self._traverse_dict(self.__dict__)
    def _traverse_dict(self,instance_dict):
        output = {}
        for key,value in instance_dict.items():
            output[key] = self._traverse(key,value)
        return output
    def _traverse(self,key,value):
        """递归的将对象转化为字典形式"""
        if isinstance(value,ToDictMixin):
            return value.to_dict()
        elif isinstance(value,dict):
            return self._traverse_dict(value)
        elif isinstance(value,list):
            return [self._traverse(key,i) for i in value]
        elif hasattr(value,'__dict__'):
            return self._traverse_dict(value.__dict__)
        else:
            return value

In [13]:
class BinaryTree(ToDictMixin):
    def __init__(self,value,left=None,right=None):
        self.value = value
        self.left = left
        self.right = right

In [14]:
tree = BinaryTree(10,
                  left=BinaryTree(7,
                                  right = BinaryTree(9)),
                  right = BinaryTree(13,
                                  left = BinaryTree(11))
                 )

In [15]:
tree.to_dict()

{'left': {'left': None,
  'right': {'left': None, 'right': None, 'value': 9},
  'value': 7},
 'right': {'left': {'left': None, 'right': None, 'value': 11},
  'right': None,
  'value': 13},
 'value': 10}

Mixin最大的优势是使用者可以随时安插这些功能,并且可以在必要的时候覆写他们,比如二叉树中节点也要求有指向父节点的引用,那么上面的树就会陷入死循环,可以在其中覆写`_traverse`方法以避免这个问题

In [ ]:
class BinaryTreeWithParent(BinaryTree):
    def __init__(self,value,left,right,partent = )